## Step1: Generating a synthetic resistivity model

In [1]:
# from dask.distributed import Client, progress
# client = Client(n_workers=10)
# client

In [2]:
import numpy as np
from discretize import TensorMesh
import discretize.utils as disutils
from SimPEG import utils

dx = 100
dy = 100
dz = 5
hx = np.ones(101) * dx
hy = np.ones(101) * dx
hz = np.ones(60) * dz
mesh =TensorMesh([hx, hy, hz], x0=[-hx.sum()/2, -hy.sum()/2, -hz.sum() + 30.])
ind_active = mesh.cell_centers[:,2]<0.

## Step 2: Run 3D EM simulation 
### 2-1 Design a flight line

In [6]:
n_sounding = 50
x = np.arange(n_sounding) * dx + -4500
y = np.r_[0.]
z = np.r_[0.]
topo = utils.ndgrid(x, y, z)
source_locations = np.c_[topo[:,:2], topo[:,2]+40.]
receiver_locations = np.c_[source_locations[:,0]+13.25, source_locations[:,1], source_locations[:,2]+2.] 

### 2-2 Genrerate a survey object

In [7]:
import SimPEG.electromagnetics.time_domain as tdem
from SimPEG import maps

In [8]:
receiver_orientation = "z"                    # "x", "y" or "z"
times = np.logspace(-5, -2, 41)               # time channels
source_radius = 10.
current_amplitude = 1.
source_list = []
n_src = topo.shape[0]

for i_src in range(n_src):
    source_location = source_locations[i_src,:]
    receiver_location = receiver_locations[i_src,:]

    receiver_list = [
        tdem.receivers.PointMagneticFluxTimeDerivative(
            receiver_location, times, orientation=receiver_orientation
        )
    ]

#     General Waveform
    
    start_time = -0.01
    peak_time = -0.005
    off_time = 0.
    triangle_waveform = tdem.sources.TriangularWaveform(
            start_time=start_time, peak_time=peak_time, off_time=off_time
    )
    triangle_waveform.has_initial_fields = True

    source_list.append(
        tdem.sources.MagDipole(
            receiver_list=receiver_list, location=source_location,
            waveform=triangle_waveform, orientation='z'
        ),
    )

# Survey
survey = tdem.Survey(source_list)

In [9]:
def generate_time_steps(
    n_constant_steps=15, increase_rate=3, start_time_step=5e-7, n_per_step=5
):
    time_steps = []
    for i_step in range(n_constant_steps):
        ht = np.ones(n_per_step) * start_time_step*increase_rate**i_step
        time_steps.append(ht)
    print("t_min:{:.1e}, t_max:{:.1e}".format(start_time_step, np.sum(time_steps)))
    return np.hstack(time_steps)

In [10]:
time_steps = generate_time_steps(n_constant_steps=13, increase_rate=2, start_time_step=5e-7, n_per_step=5)

t_min:5.0e-07, t_max:2.0e-02


In [11]:
active_map = maps.InjectActiveCells(mesh, indActive=ind_active, valInactive=1e-8)
exp_map = maps.ExpMap()

In [12]:
mapping = active_map * exp_map

In [13]:
from simpegaem import GlobalSimulationAEM
sim = GlobalSimulationAEM(
    mesh=mesh, 
    topo=topo, 
    sigmaMap=mapping, 
    survey=survey, 
    time_steps=time_steps, 
    parallel_option='dask'
)

>> Use dask for parallelization


In [14]:
nP = ind_active.sum()
m_true = np.ones(nP) * np.log(1./100.)

In [16]:
%%time
dobs = sim.dpred(m_true)

CPU times: user 56.4 ms, sys: 155 ms, total: 211 ms
Wall time: 16.2 s


In [ ]:
%%time
J_sigma = sim.getJ_sigma(m_true)

In [35]:
# from discretize import tests
# nP = ind_active.sum()
# m_true = np.ones(nP) * np.log(1./100.)
# m_ini = np.ones(nP) * np.log(1./20.)

# dobs = sim.dpred(m_true)
# uncert = 0.03*abs(dobs)
# dini = sim.dpred(m_ini)
# dr = (dini - dobs) / uncert

# def misfit(m, dobs):
#     dpred = sim.dpred(m)
#     misfit = 0.5 * np.linalg.norm((dpred - dobs) / uncert) ** 2
#     dmisfit = sim.Jtvec(m, dr / uncert)
#     return misfit, dmisfit

# def derChk(m):
#     return misfit(m, dobs)

# passed = tests.check_derivative(derChk, m_ini, num=4, plotIt=False, eps=1e-27)